### **4.13 - [Práctica] Retry and Alerting**

Vamos a utilizar este código para este capítulo:

<center><img src="https://i.postimg.cc/ZnTwR1HQ/a715.png"></center>

___

### **DAGs**

#### **`dagrun_timeout`**

Vamos a ejecutar nuestro DAG por primera vez:

<center><img src="https://i.postimg.cc/GmBqHPPb/a716.png"></center>

Haz clic en el DAG. A partir de ahí, si pasas por encima del primer círculo y miras la diferencia entre la start_date  y la end_date, podemos ver que el primer DAGRun tardó unos 45 segundos en terminar aproximadamente.

<center><img src="https://i.postimg.cc/kXgjdPB6/a717.png"></center>

Sólo para asegurarnos, vamos a ejecutar un segundo DAG Run. Bien, ahora volvamos a la Tree view, y de nuevo, al pasar por el segundo círculo, el DAGRun tardó alrededor de 45 segundos en terminar. Basándonos en estos resultados, podemos decir, que un DAGRun no debería tomar más de 1:15 (75 segundos) minutos para tener éxito. Añadimos un timeout ya que puede tardar más o menos de 45 segundos en procesarse.

<center><img src="https://i.postimg.cc/PqYyMtFc/a718.png"></center>

En nuestro código vamos a agregar el parámetro **`dagrun_timeout`** en 75 segundos:

<center><img src="https://i.postimg.cc/cHccVFWB/a719.png"></center>

Ahora ejecutamos un tercer DAG Run. En la Tree view, el tiempo que tarda el DAGRun sigue siendo inferior a 75 segundos, así que todo está bien. 

<center><img src="https://i.postimg.cc/4N2Wf298/a720.png"></center>

Ahora, vamos a cambiar el timeout que fijamos en 25 segundos.

<center><img src="https://i.postimg.cc/DZhgjt9z/a721.png"></center>

Lanzamos un cuarto DAG Run. Vaya a la Tree view y espere hasta que el nuevo DAGRun haya terminado. Y falló. ¿Por qué? Porque tarda unos 45 segundos en terminar, que es más que el nuevo tiempo de espera (timeout) que hemos fijado para los DAGRuns.

<center><img src="https://i.postimg.cc/dVznd7nr/a722.png"></center>

Hay que recordar algunas cosas. En primer lugar, aunque este DAGRun haya fallado, el siguiente podrá seguir ejecutándose. No hay una forma estándar de evitar que Airflow programe una DAGRun si la anterior ha fallado. Además, si estableces una de las tareas con el parámetro '**`depends_on_past`**', esta tarea podrá seguir ejecutándose para el siguiente DAGRun ya que su instancia anterior tuvo éxito aunque el DAGRun esté marcado como fallido. Finalmente, si recuerdas, dije que el parámetro 'dagrun_timeout' se aplica si el número de DAGRuns activos es igual al número de '**`max_active_runs_per_dag`**'. Ahora, si echamos un vistazo rápidamente al valor de este parámetro en airflow.cfg, podemos ver que está establecido en 16. Entonces, ¿por qué ha funcionado? Bueno, porque estamos utilizando el **`Sequential executor`** y **`Airflow automáticamente baja este valor a 1`**.

<center><img src="https://i.postimg.cc/4yN1ZpNk/a723.png"></center>

___

#### **`on_failure_callback`**
#### **`on_success_callback`**

Además de tener un timeout, quizás quisieras o hacer algo si el DAGRun ha fallado o ha tenido éxito. Bueno, podríamos añadir los callbacks **`on_failure_callback`** así como **`on_success_callback`**.

Hagamos esto. Justo encima de la definición del DAG, creamos dos funciones denominadas respectivamente **`on_success_dag`** y **`on_failure_dag`** pero puedes darle los nombres que quieras. **`No olvides que estos dos callbacks toman un diccionario de contexto en parámetros que vamos a mostrar en la salida estándar añadiendo una instrucción print para cada uno`**. Podemos añadir otro print para saber a qué callback se llama. Finalmente, añadimos los parámetros on_success_callback y on_failure_callback a la definición del DAG que asignamos a las funciones que acabamos de hacer.

<center><img src="https://i.postimg.cc/dVTjqsv1/a724.png"></center>

Lanzamos un nuevo DAG Run. Y como era de esperar el DAGRun falló. Bien, eso es genial, pero ¿qué pasa con las callbacks? Bueno, para comprobar si la función 'on_failure_dag' ha sido llamada, necesitamos echar un vistazo a los scheduler logs. Ve a tu terminal y escribe "docker ps". A continuación, abre una bash session against the docker container con el comando "docker exec -it" copia y pega el id del contenedor de airflow, y luego "/bin/bash". Enter.

<center><img src="https://i.postimg.cc/4Ny1SvSp/a725.png"></center>
<center><img src="https://i.postimg.cc/LXmtBjPY/a726.png"></center>

Ahora estamos dentro del contenedor, escribe "cd" para entrar en la carpeta de inicio. Luego "cd logs/scheduler/. Escribe "ls". Aquí encontrarás los logs del scheduler producidos por día, donde cada día crea una carpeta y tienes la última carpeta para obtener el último día de logs. Entra en la carpeta. Volvemos a teclear "ls" y obtenemos los diferentes archivos de logs correspondientes a nuestros DAGs. Como hemos utilizado el DAG alert_dag, vamos a abrir el archivo alert_dag.py.log y buscar la palabra "Failure" que debe imprimir nuestro callback. Escribe "cat alert_dag.py.log | grep failure". Enter. Y tenemos la línea. Así que nuestro callback 'on_failure_dag' ha sido bien llamado por el scheduler.

<center><img src="https://i.postimg.cc/5y08D1nf/a727.png"></center>
<center><img src="https://i.postimg.cc/T14n6S4g/a728.png"></center>

En realidad, si escribimos el comando "cat alert_dag.py.log" y nos desplazamos hacia arriba hasta encontrar la palabra Failure, justo debajo, puedes ver el contenido del **`"diccionario de contexto"`** dado en los parámetros de nuestros callbacks. Es básicamente toda la información del DAGRun fallido.

<center><img src="https://i.postimg.cc/xjxvMJ18/a729.png"></center>
<center><img src="https://i.postimg.cc/RVWcRSWR/a730.png"></center>

___

### **Tasks**

#### **`retries`**

Antes de avanzar, cambia el dagrun_timeout de 25 a 75 segundos. Entonces, en este punto hemos visto cómo ser advertidos cuando un DAGRun falló, pero ¿qué pasa si una tarea falló? Empecemos por añadir el parámetro **`retries`**. Dependiendo de dónde quieras aplicarlo, puedes definirlo en los argumentos por defecto para que todas las tareas tengan la misma configuración de reintento (retry settings), o puedes definirlo en la definición del operator para que sólo esa tarea sea reintentada. La elección de la primera forma o la segunda depende realmente de tu caso de uso. En ningún caso vamos a quedarnos con los argumentos por defecto.

<center><img src="https://i.postimg.cc/4xtVDWbH/a731.png"></center>

A continuación, cambiamos el bash_command de la tarea t1 para que el scheduler piense que la tarea ha fallado. Elimine el comando "echo" y escriba "exit 1". Como sabrás, cuando un programa devuelve un valor distinto de 0, significa que ese programa ha fallado. Eso es lo que estamos simulando al devolver 1 desde el BashOperator.

Vamos a volver a ejecutar un DAG Run. En algún momento, debería empezar a ver que la primera tarea se ejecuta y luego se reintenta, como se muestra en el círculo en naranja aquí. El número aquí corresponde al número de reintentos realizados. **`Como hemos definido el número de reintentos en tres (retries = 3), la tarea será reintentada tres veces como máximo. Como la tarea t1 siempre fallará, después del tercer reintento, la tarea se marcará como fallida y el DAGRun también`**. Eso es lo que obtenemos aquí.

<center><img src="https://i.postimg.cc/5tSv0jrx/a732.png"></center>
<center><img src="https://i.postimg.cc/pTVjnRNK/a733.png"></center>

___

#### **`emails`**
#### **`email_on_failure`**
#### **`email_on_retry`**

Bien, ya hemos visto cómo reintentar una tarea, vamos a añadir una dirección de correo electrónico para que podamos recibir un email cuando una tarea no haya tenido éxito. En los argumentos por defecto, añadimos la key "emails" con una lista de emails. Por ejemplo, owner@test.com. Si realmente quieres recibir un email en caso de fallo o reintento, tienes que poner tu dirección de email así como configurar el servidor SMTP en airflow.cfg como hicimos en el pipeline de datos FOREX. Por defecto los argumentos 'email_on_failure' y 'email_on_retry' están configurados a True. Podemos añadirlos en el diccionario así, y sólo mantener 'email_on_failure' establecido a True.

<center><img src="https://i.postimg.cc/hjCdL6g6/a734.png"></center>

___

#### **`execution_timeout`**
#### **`on_failure_callback`**
#### **`on_success_callback`**

Tengo un pequeño ejercicio para ti. Quiero que definas el parámetro 'execution_timeout' y que implementes los callbacks 'on_failure_callback' y 'on_success_callback'. Eso es exactamente lo que hicimos a nivel del DAG excepto que esta vez debes definir todo en los argumentos por defecto y crear dos nuevas funciones.

Primero creamos dos nuevas funciones que llamaremos "on_success_task" y "on_failure_task". De nuevo, no olvides que el contexto dado en los parámetros trae información sobre la instancia de la tarea. Entonces voy a imprimir el parámetro de contexto, pero puedes hacer lo que quieras en estas callbacks. Como hicimos para el DAG, también imprimimos si estamos en el success o failure callback. Yo sólo imprimo información, pero como usamos Python, podríamos usar un módulo y notificar a un sistema externo como una base de datos si quieres. Como mejor práctica, nuestros callbacks deberían ser muy ligeros, como empujar una notificación o limpiar un estado. No debemos poner un proceso pesado, ya que, podría ralentizar nuestra aplicación. Realmente depende de nosotros y de nuestro caso de uso el saber qué debemos implementar. A continuación, en los argumentos por defecto (default_args), añadimos los parámetros 'on_failure_callback' y 'on_success_callback' y asignamos sus correspondientes funciones 'on_failure_task' y 'on_success_task'. Por último, añadimos el parámetro 'execution_timeout' con un objeto timedelta de 60 segundos. Fíjate que no es recomendable utilizar execution timeout así, ya que cada tarea puede tener un execution time diferente. Es mejor utilizar este parámetro cuando se define la tarea en la definición del operator. Muy bien, ahora eres capaz de reintentar una tarea y ser avisado cuando algo va mal.

<center><img src="https://i.postimg.cc/QCHW0mdt/a735.png"></center>